# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/Parolin/Documents/AEco")

aircraft_type = 'cargo'  #pax or cargo

input_path = './Data/mistral_inputs.xlsx'
input_sheet = 'Bravo'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_mistral.xlsx'

iterations = 1000000
chunks = 2000  #'auto' or more than 500

In [2]:
from Tools import *
from Model import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

In [5]:
p.data

<xarray.Dataset>
Dimensions:             (i: 2000)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 5 ... 1994 1995 1996 1997 1998 1999
Data variables:
    takt                (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    OEW                 (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    flights_year        (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    FH                  (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    loadfactor          (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    payload             (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    d_f                 (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    lifetime            (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    fleet               (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    grid_wind           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    grid_gas            (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    grid_hydro          (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    E_office            (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    water_office        (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    wastewater_office   (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    paper_use           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    devmonths           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    developers          (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    car_travel          (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    air_travel          (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    new_factory         (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    new_machine         (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    prototypes          (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    ironbirds           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    test_FH             (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    p_Al                (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    p_steel             (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    p_CFRP              (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    p_GFRP              (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    p_inconel           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    p_Ti                (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    b2f_Al              (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    b2f_steel           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    b2f_CFRP            (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    b2f_GFRP            (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    b2f_inconel         (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    b2f_Ti              (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    reuse               (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    E_factory           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    water_factory       (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    wastewater_factory  (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    lubricant           (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    d_lorry             (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    d_air               (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    d_sea               (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    m_lorry             (i) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    m_air               (i) float64 dask.array<chunksize=(100,), met

Then, we run the function that calculates PKM for the study.

In [5]:
p = pkm(aircraft_type, p)

<br></br>
Executing the UP reading and writing functions.

In [6]:
UP_dataframe = read_unit_processes(database_path)

In [7]:
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [8]:
inventory = LCI(name=input_sheet, type=aircraft_type, iterations=iterations, UP=UP, parameters=p)

In [9]:
inventory.run()

,Array,Chunk
Bytes,14.68 GB,29.36 MB
Shape,"(1835, 1000000)","(1835, 2000)"
Count,44008 Tasks,500 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,14.68 GB,29.36 MB
Shape,"(1835, 1000000)","(1835, 2000)"
Count,15501 Tasks,500 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [10]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

In [15]:
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [16]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [17]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconium-95'
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'urban air close to ground' ... 'non-urban air or from high stacks'
Data variables:
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 1.0 0.0456 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.56 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    PMF             (Substances) float64 0.0 0.0 0.22 0.0 ... 0.0 0.0 0.0 0.0
    TET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    NLT             (Substances) float64 -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 -0.0
    OD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ME              (Substances) float64 0.0 0.0 0.039 0.0 ... 0.0 0.0 0.0 0.0
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [18]:
aeco = LCIA.build(inventory, CF)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [19]:
display(aeco.MP)

,Array,Chunk
Bytes,144.00 MB,288.00 kB
Shape,"(1000000, 18)","(2000, 18)"
Count,47509 Tasks,500 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,144.00 MB,288.00 kB
Shape,"(1000000, 18)","(2000, 18)"
Count,19002 Tasks,500 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [22]:
aeco.mean("MP", by='sum')

<xarray.DataArray 'stack-58e3c83ec7eb2e0f1860b567baaf84c3' (Categories: 18)>
dask.array<mean_agg-aggregate, shape=(18,), dtype=float64, chunksize=(18,), chunktype=numpy.ndarray>
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [23]:
display(aeco.EP)

,Array,Chunk
Bytes,24.00 MB,48.00 kB
Shape,"(1000000, 3)","(2000, 3)"
Count,51010 Tasks,500 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,24.00 MB,48.00 kB
Shape,"(1000000, 3)","(2000, 3)"
Count,22503 Tasks,500 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [24]:
aeco.mean("EP", by="sum")

<xarray.DataArray 'stack-86358ac9961013e9926ea690a0a4996a' (AOP: 3)>
dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [25]:
aeco.build_CTV(parameterset=p)
display(aeco.CTV)

KeyError: "not all values found in index 'Parameters'"

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [70]:
aeco.save(output_path, LCI=False)

LCIA saved at .\results\Bravo_outputs


In [ ]:
#aeco.to_excel(output_path)